#### Train doc2vec using parsed wiki10 documents (wikipedia article text with multiple class labels)

In [148]:
# import modules & set up logging
import gensim
from gensim.models.doc2vec import TaggedDocument
import logging
from pathlib import Path
import re
import string 
from itertools import islice
import multiprocessing
from gensim.models import Doc2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [169]:
DATA_DIR = Path('../../data/wiki10')
TEXT_DIR = DATA_DIR / 'text' 
MODEL_PATH = DATA_DIR / 'doc2vec.model'

Gensim's implementation of Doc2Vec requires as input:
* Iterable of TaggedDocument objects
* TaggedDocument corresponds to a single sentence and contains a list of tokens in that sentence, and a list of one (or more) labels used to identify that document


Implementing iterator to parse our input file, and crudely split into sentences of tokens 

**N.B.**
* We cannot use a generator as training will iterate over input multiple times

In [9]:
class Corpus2TaggedDocument(object):
    
    def __init__(self, input_path):
        self.input_path = Path(input_path)
        
    def __iter__(self):
        ''' yield list of tokens for a single sentence in corpus '''
        for document_path in self.input_path.iterdir():
            document = document_path.read_text().lower()
            for sentence in self.sentencize(document):
                yield TaggedDocument(words=self.tokenize(sentence), tags=[document_path.name])
       
    def sentencize(self, text):
        return text.split('. ')
    
    def tokenize(self, text):
        return re.sub(f'([{string.punctuation}])', r' \1 ', text).split()

#### Parsing input

In [10]:
sentences = Corpus2TaggedDocument(DATA_DIR)

Example sentences

In [11]:
for sentence in islice(sentences, 5): print(sentence, end='\n\n')

TaggedDocument(['miso', 'soup', '(', '味噌汁', ',', 'miso', 'shiru', '?', ')', 'is', 'a', 'traditional', 'japanese', 'soup', 'consisting', 'of', 'a', 'stock', 'called', '"', 'dashi', '"', 'into', 'which', 'is', 'mixed', 'softened', 'miso', 'paste'], ['33d29682601cfaa391fe7516d90d6baf'])

TaggedDocument(['although', 'the', 'suspension', 'of', 'miso', 'paste', 'into', 'dashi', 'is', 'the', 'only', 'characteristic', 'that', 'actually', 'defines', 'miso', 'soup', ',', 'many', 'other', 'ingredients', 'are', 'added', 'depending', 'on', 'regional', 'and', 'seasonal', 'recipes', ',', 'and', 'personal', 'preference'], ['33d29682601cfaa391fe7516d90d6baf'])

TaggedDocument(['the', 'choice', 'of', 'miso', 'paste', 'for', 'the', 'soup', 'defines', 'a', 'great', 'deal', 'of', 'its', 'character', 'and', 'flavor'], ['33d29682601cfaa391fe7516d90d6baf'])

TaggedDocument(['most', 'miso', 'pastes', 'can', 'be', 'categorized', 'into', 'red', '(', 'akamiso', ')', ',', 'white', '(', 'shiromiso', ')', ',', 'or',

Number of sentences in corpus

In [12]:
sum(1 for _ in sentences)

1511872

In [13]:
cores = multiprocessing.cpu_count()

#### Training model

Training doc2vec model using a window of 5 surrounding tokens for word vectors, and minimum token frequency of 2. Will see what sort of result we get with this simple setup, then try tuning parameters:

* We are using default algorithm "DBOW", if we later use the alternative "DM" algorithm we may want test using full documents instead of sentences as our training data [link](https://groups.google.com/forum/#!topic/gensim/2QkGx2C1pNQ)
* Optimising learning rate  [link](https://rare-technologies.com/doc2vec-tutorial/)
* General parameter tuning [link](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-IMDB.ipynb)
* Paper describing effects of document length [link](https://arxiv.org/pdf/1607.05368.pdf)

In [ ]:
model = Doc2Vec(
    sentences,
    size=100,
    min_count=2, 
    window=5,
    workers=cores
    )

2018-02-06 06:18:05,412 : INFO : collecting all words and their counts
2018-02-06 06:18:05,434 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-02-06 06:18:06,347 : INFO : PROGRESS: at example #10000, processed 326436 words (361678/s), 24960 word types, 120 tags
2018-02-06 06:18:07,250 : INFO : PROGRESS: at example #20000, processed 648630 words (357775/s), 36480 word types, 240 tags
2018-02-06 06:18:08,154 : INFO : PROGRESS: at example #30000, processed 971417 words (358060/s), 46077 word types, 365 tags
2018-02-06 06:18:09,067 : INFO : PROGRESS: at example #40000, processed 1287076 words (346842/s), 54771 word types, 495 tags
2018-02-06 06:18:09,945 : INFO : PROGRESS: at example #50000, processed 1588239 words (344193/s), 61737 word types, 615 tags
2018-02-06 06:18:10,846 : INFO : PROGRESS: at example #60000, processed 1897401 words (344463/s), 69040 word types, 757 tags
2018-02-06 06:18:11,752 : INFO : PROGRESS: at example #70000, processed 220908

2018-02-06 06:19:03,410 : INFO : PROGRESS: at example #640000, processed 20595333 words (351447/s), 269676 word types, 8393 tags
2018-02-06 06:19:04,291 : INFO : PROGRESS: at example #650000, processed 20908102 words (356032/s), 271777 word types, 8512 tags
2018-02-06 06:19:05,162 : INFO : PROGRESS: at example #660000, processed 21217257 words (356177/s), 273934 word types, 8639 tags
2018-02-06 06:19:06,047 : INFO : PROGRESS: at example #670000, processed 21527911 words (355221/s), 276179 word types, 8766 tags
2018-02-06 06:19:06,915 : INFO : PROGRESS: at example #680000, processed 21833459 words (353256/s), 278552 word types, 8883 tags
2018-02-06 06:19:07,836 : INFO : PROGRESS: at example #690000, processed 22156692 words (356903/s), 281204 word types, 9023 tags
2018-02-06 06:19:08,754 : INFO : PROGRESS: at example #700000, processed 22485475 words (359247/s), 283328 word types, 9117 tags
2018-02-06 06:19:09,641 : INFO : PROGRESS: at example #710000, processed 22798885 words (357815/s

2018-02-06 06:20:01,277 : INFO : PROGRESS: at example #1270000, processed 40980666 words (349795/s), 406251 word types, 16517 tags
2018-02-06 06:20:02,209 : INFO : PROGRESS: at example #1280000, processed 41291272 words (334095/s), 407936 word types, 16664 tags
2018-02-06 06:20:03,139 : INFO : PROGRESS: at example #1290000, processed 41597434 words (330190/s), 409688 word types, 16787 tags
2018-02-06 06:20:04,069 : INFO : PROGRESS: at example #1300000, processed 41919682 words (347700/s), 411802 word types, 16931 tags
2018-02-06 06:20:05,016 : INFO : PROGRESS: at example #1310000, processed 42246580 words (346262/s), 413679 word types, 17081 tags
2018-02-06 06:20:05,926 : INFO : PROGRESS: at example #1320000, processed 42569029 words (355621/s), 415353 word types, 17212 tags
2018-02-06 06:20:06,841 : INFO : PROGRESS: at example #1330000, processed 42886107 words (347394/s), 417134 word types, 17340 tags
2018-02-06 06:20:07,746 : INFO : PROGRESS: at example #1340000, processed 43206297 

2018-02-06 06:21:20,813 : INFO : PROGRESS: at 1.89% examples, 81487 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:21,831 : INFO : PROGRESS: at 1.94% examples, 81526 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:22,936 : INFO : PROGRESS: at 1.99% examples, 81556 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:23,940 : INFO : PROGRESS: at 2.03% examples, 81606 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:24,981 : INFO : PROGRESS: at 2.08% examples, 81595 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:26,001 : INFO : PROGRESS: at 2.12% examples, 81612 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:27,032 : INFO : PROGRESS: at 2.17% examples, 81643 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:28,052 : INFO : PROGRESS: at 2.21% examples, 81661 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:29,194 : INFO : PROGRESS: at 2.27% examples, 81649 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:21:30,247 : INFO : PROGRESS: at 2.31% examples, 81620 words/s, in_qsize 5, ou

2018-02-06 06:22:48,617 : INFO : PROGRESS: at 5.71% examples, 81586 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:49,621 : INFO : PROGRESS: at 5.75% examples, 81546 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:50,629 : INFO : PROGRESS: at 5.80% examples, 81565 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:51,634 : INFO : PROGRESS: at 5.84% examples, 81578 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:52,769 : INFO : PROGRESS: at 5.89% examples, 81520 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:53,781 : INFO : PROGRESS: at 5.92% examples, 81361 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:54,788 : INFO : PROGRESS: at 5.96% examples, 81268 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:55,810 : INFO : PROGRESS: at 6.00% examples, 81223 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:56,856 : INFO : PROGRESS: at 6.05% examples, 81219 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:22:57,939 : INFO : PROGRESS: at 6.10% examples, 81248 words/s, in_qsize 5, ou

...

2018-02-06 06:42:48,884 : INFO : PROGRESS: at 61.86% examples, 86917 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:49,897 : INFO : PROGRESS: at 61.92% examples, 86921 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:50,913 : INFO : PROGRESS: at 61.97% examples, 86923 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:52,127 : INFO : PROGRESS: at 62.02% examples, 86924 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:53,187 : INFO : PROGRESS: at 62.08% examples, 86924 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:54,256 : INFO : PROGRESS: at 62.12% examples, 86924 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:55,279 : INFO : PROGRESS: at 62.18% examples, 86932 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:56,401 : INFO : PROGRESS: at 62.23% examples, 86928 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:57,425 : INFO : PROGRESS: at 62.26% examples, 86908 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:42:58,552 : INFO : PROGRESS: at 62.31% examples, 86898 words/s, in_q

2018-02-06 06:44:14,169 : INFO : PROGRESS: at 66.02% examples, 87172 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:15,181 : INFO : PROGRESS: at 66.07% examples, 87180 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:16,238 : INFO : PROGRESS: at 66.12% examples, 87180 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:17,258 : INFO : PROGRESS: at 66.17% examples, 87183 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:18,285 : INFO : PROGRESS: at 66.23% examples, 87190 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:19,294 : INFO : PROGRESS: at 66.28% examples, 87187 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:20,416 : INFO : PROGRESS: at 66.33% examples, 87194 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:21,460 : INFO : PROGRESS: at 66.39% examples, 87200 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:22,524 : INFO : PROGRESS: at 66.44% examples, 87205 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:44:23,608 : INFO : PROGRESS: at 66.49% examples, 87209 words/s, in_q

2018-02-06 06:45:39,240 : INFO : PROGRESS: at 70.22% examples, 87440 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:40,283 : INFO : PROGRESS: at 70.27% examples, 87446 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:41,331 : INFO : PROGRESS: at 70.31% examples, 87446 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:42,361 : INFO : PROGRESS: at 70.37% examples, 87452 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:43,449 : INFO : PROGRESS: at 70.42% examples, 87455 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:44,495 : INFO : PROGRESS: at 70.47% examples, 87461 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:45,508 : INFO : PROGRESS: at 70.51% examples, 87467 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:46,530 : INFO : PROGRESS: at 70.57% examples, 87473 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:47,552 : INFO : PROGRESS: at 70.61% examples, 87475 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:45:48,637 : INFO : PROGRESS: at 70.67% examples, 87478 words/s, in_q

2018-02-06 06:47:05,000 : INFO : PROGRESS: at 74.34% examples, 87611 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:06,042 : INFO : PROGRESS: at 74.39% examples, 87611 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:07,077 : INFO : PROGRESS: at 74.44% examples, 87617 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:08,230 : INFO : PROGRESS: at 74.48% examples, 87596 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:09,313 : INFO : PROGRESS: at 74.52% examples, 87584 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:10,380 : INFO : PROGRESS: at 74.57% examples, 87588 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:11,396 : INFO : PROGRESS: at 74.62% examples, 87590 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:12,453 : INFO : PROGRESS: at 74.67% examples, 87594 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:13,516 : INFO : PROGRESS: at 74.71% examples, 87588 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:47:14,525 : INFO : PROGRESS: at 74.76% examples, 87589 words/s, in_q

2018-02-06 06:48:31,061 : INFO : PROGRESS: at 78.53% examples, 87789 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:32,105 : INFO : PROGRESS: at 78.58% examples, 87794 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:33,136 : INFO : PROGRESS: at 78.63% examples, 87799 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:34,365 : INFO : PROGRESS: at 78.68% examples, 87794 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:35,436 : INFO : PROGRESS: at 78.74% examples, 87797 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:36,470 : INFO : PROGRESS: at 78.79% examples, 87797 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:37,529 : INFO : PROGRESS: at 78.84% examples, 87801 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:38,666 : INFO : PROGRESS: at 78.90% examples, 87806 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:39,737 : INFO : PROGRESS: at 78.95% examples, 87810 words/s, in_qsize 5, out_qsize 0
2018-02-06 06:48:40,802 : INFO : PROGRESS: at 79.01% examples, 87813 words/s, in_q

Saving model / document vectors

In [37]:
model.save(str(MODEL_PATH)) 

2018-02-06 18:24:12,863 : INFO : saving Doc2Vec object under ../../data/wiki10/doc2vec.model, separately None
2018-02-06 18:24:12,871 : INFO : storing np array 'syn0' to ../../data/wiki10/doc2vec.model.wv.syn0.npy
2018-02-06 18:24:13,006 : INFO : not storing attribute syn0norm
2018-02-06 18:24:13,010 : INFO : storing np array 'syn1neg' to ../../data/wiki10/doc2vec.model.syn1neg.npy
2018-02-06 18:24:13,197 : INFO : not storing attribute cum_table
2018-02-06 18:24:16,413 : INFO : saved ../../data/wiki10/doc2vec.model
